In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sklearn
sklearn.__version__

'0.20.2'

In [3]:
import pandas as pd
import pickle as pkl

In [4]:
%run utils.ipynb
%run targets.ipynb

In [5]:
file = 'ETHBTC_1m_1519496760000_1549568220000.csv'

In [6]:
# Get Data
df = pd.read_csv('../data/'+file)
df.shape
df.head(10)

(500000, 12)

,OPEN_TIME,OPEN,HIGH,LOW,CLOSE,VOLUME,CLOSE_TIME,QUOTE_VOL,NUM_TRADES,TAKE_BASE_VOL,TAKE_QUOTE_VOL,EXTRA
0,1519496760000,0.085541,0.085570,0.085407,0.085473,27.714,1519496819999,2.370345,192,15.308,1.309733,0
1,1519496820000,0.085473,0.085570,0.085448,0.085449,20.128,1519496879999,1.720609,122,6.579,0.562654,0
2,1519496880000,0.085448,0.085569,0.085400,0.085538,45.510,1519496939999,3.890176,209,19.426,1.660960,0
3,1519496940000,0.085538,0.085546,0.085402,0.085540,22.231,1519496999999,1.900513,105,14.320,1.224483,0
4,1519497000000,0.085539,0.085552,0.085464,0.085464,15.243,1519497059999,1.303412,117,8.373,0.716219,0
5,1519497060000,0.085466,0.085555,0.085464,0.085545,31.895,1519497119999,2.727413,131,19.237,1.645547,0
6,1519497120000,0.085545,0.085545,0.085464,0.085539,13.325,1519497179999,1.139346,78,7.311,0.625352,0
7,1519497180000,0.085539,0.085555,0.085414,0.085517,28.202,1519497239999,2.411093,195,14.409,1.232346,0
8,1519497240000,0.085460,0.085545,0.085419,0.085534,15.302,1519497299999,1.307959,134,7.216,0.617035,0
9,1519497300000,0.085440,0.085545,0.085440,0.085533,13.730,1519497359999,1.173580,105,6.186,0.529001,0


In [7]:
data = df.loc[:, ['OPEN_TIME', 'CLOSE_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.loc[:, ['OPEN', 'HIGH', 'LOW', 'CLOSE']] = data.loc[:, ['OPEN', 'HIGH', 'LOW', 'CLOSE']].diff()
data.shape
data[:5]

(500000, 6)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE
0,1519496760000,1519496819999,NaN,NaN,NaN,NaN
1,1519496820000,1519496879999,-0.000068,0.000000e+00,0.000041,-0.000024
2,1519496880000,1519496939999,-0.000025,-1.000000e-06,-0.000048,0.000089
3,1519496940000,1519496999999,0.000090,-2.300000e-05,0.000002,0.000002
4,1519497000000,1519497059999,0.000001,6.000000e-06,0.000062,-0.000076


In [8]:
# Get Target
%run targets.ipynb
target = get_target_1('../data/'+file)
target.shape
target.head()

(499999, 2)

,CLOSE_TIME,TARGET
0,1519496819999,1.0
1,1519496879999,0.0
2,1519496939999,0.0
3,1519496999999,1.0
4,1519497059999,0.0


In [9]:
data = pd.merge(data, target, on='CLOSE_TIME')
data = data.iloc[1:]
data.shape
data.head()

(499998, 7)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE,TARGET
1,1519496820000,1519496879999,-0.000068,0.000000e+00,0.000041,-0.000024,0.0
2,1519496880000,1519496939999,-0.000025,-1.000000e-06,-0.000048,0.000089,0.0
3,1519496940000,1519496999999,0.000090,-2.300000e-05,0.000002,0.000002,1.0
4,1519497000000,1519497059999,0.000001,6.000000e-06,0.000062,-0.000076,0.0
5,1519497060000,1519497119999,-0.000073,3.000000e-06,0.000000,0.000081,1.0


In [10]:
TR_split, CV_split, TT_split = TR_CV_TT_split(data, 0.8, 0, 0.2)

TR_split.shape
CV_split.shape
TT_split.shape

TR_split.head()

(399998, 7)

(0, 7)

(100000, 7)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE,TARGET
1,1519496820000,1519496879999,-0.000068,0.000000e+00,0.000041,-0.000024,0.0
2,1519496880000,1519496939999,-0.000025,-1.000000e-06,-0.000048,0.000089,0.0
3,1519496940000,1519496999999,0.000090,-2.300000e-05,0.000002,0.000002,1.0
4,1519497000000,1519497059999,0.000001,6.000000e-06,0.000062,-0.000076,0.0
5,1519497060000,1519497119999,-0.000073,3.000000e-06,0.000000,0.000081,1.0


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
window = 30
def data_transformer(id, data, target, retrain=False):
    global scaler
    global window
    
    if retrain==True:
        scaler = scaler.fit(data)
        
    data = scaler.transform(data)
    return get_multiple_ts_dataset(id, data, target, tr_win=window, tt_win=0, point_target=True)

In [12]:
TR_id     = TR_split.iloc[:, :2  ].values
TR_data   = TR_split.iloc[:, 2:-1].values
TR_target = TR_split.iloc[:, -1: ].values

TT_id     = TT_split.iloc[:, :2  ].values
TT_data   = TT_split.iloc[:, 2:-1].values
TT_target = TT_split.iloc[:, -1: ].values

TR_id.shape
TR_data.shape
TR_target.shape

TT_id.shape
TT_data.shape
TT_target.shape

(399998, 2)

(399998, 4)

(399998, 1)

(100000, 2)

(100000, 4)

(100000, 1)

In [13]:
%run hyperopt.ipynb
%run models.ipynb

In [14]:
hyperopt = Hyperopt(RFC())
best_loss, best_params = hyperopt.run(
    TR_id, TR_data, TR_target, 
    eval_set={
        'TT': (TT_id, TT_data, TT_target)
    },
    n_cv=3,
    algo='grid',
    data_transformer=data_transformer,
    max_evals=1
)

Hyperopt Iteration : 1
{'n_estimators': 10, 'min_samples_split': 5, 'criterion': 'entropy', 'max_features': 'sqrt'}


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:32: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   36.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.2s finished


('CV_fold : ', 0, 49.6162269115448)
         acc       auc        f1   log_loss
TT  0.502096  0.501547  0.484998  17.197207
TR  0.985265  0.985383  0.984661   0.508944
CV  0.502139  0.502051  0.492426  17.195725


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.0min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    1.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.2s finished


('CV_fold : ', 1, 77.52828812599182)
         acc       auc        f1   log_loss
TT  0.501625  0.501434  0.490349  17.213450
TR  0.985537  0.985638  0.984968   0.499538
CV  0.499869  0.499734  0.488478  17.274130


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   57.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    1.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.2s finished


('CV_fold : ', 2, 72.6518120765686)
         acc       auc        f1   log_loss
TT  0.499675  0.499640  0.490911  17.280823
TR  0.985449  0.985552  0.984886   0.502592
CV  0.500094  0.499663  0.481868  17.266356


In [ ]:
from sklearn.ensemble import RandomForestClassifier

TR_id, TR_X, TR_y = data_transformer(TR_id, TR_data, TR_target, retrain=True)
TT_id, TT_X, TT_y = data_transformer(TT_id, TT_data, TT_target, retrain=False)

model = RandomForestClassifier(**best_params)
model = model.fit(TR_X, TR_y)

TR_y_pred = model.predict_proba(TR_X)[:, 1]
TT_y_pred = model.predict_proba(TT_X)[:, 1]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(TT_metrics, name='TT'))

print(res)

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

In [ ]:
data_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'intv':'1m', 
    'cols':'OHCL',
    'win':window
}.iteritems()])

proc_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'model': nb_name.replace('.ipynb', ''),
    'TT_accuracy':'{:.4f}'.format(TT_metrics['acc']),
    'CV_accuracy':'{:.4f}'.format(best_loss)
}.iteritems()])

import time
time_tag = time.strftime('%y%m%d_%H%M%S')

big_tag = time_tag+'_|_'+data_tag+'_|_'+proc_tag

data_dump = {
    'best_loss': best_loss,
    'best_params': best_params,
    'model': model,
    'TR_id': TR_id,
    'TR_y_pred': TR_y_pred,
    'TT_id': TT_id,
    'TT_y_pred': TT_y_pred,
    'results': res
}

f = open('../models/'+time_tag+'.pkl', 'wb')
pkl.dump(data_dump, f, -1)
f.close()

%notebook -e ../models/{big_tag}.ipynb